### 환경 install

### Ensemble 검색 방식의 경우, 벡터 검색 + 전문 검색을 함께 활용하는 방식입니다. 
### 해당 코드에서는 기본 벡터 검색으로 문서 5개, 전문 검색 5개, 가중치 0.5, 0.5로 지정,
### 테스트 시 가중치 및 개수 변경 가능합니다. (테스트용으로 15개, 15개로 현재 되어있음

In [1]:
# pip install langchain-google-genai
# pip install langchain langchain-community langchain-core langchain-text-splitters langchain-google-genai faiss-cpu pymupdf
# pip install langchain-postgres psycopg2-binary
# pip install google-generativeai
# pip install langchain langchain-community langchain-openai
# pip install langchain-openai
# pip install kiwipiepy kiwipiepy-model

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.vectorstores import FAISS
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import PGVector
# # from langchain_community.retrievers import MergerRetriever  

# # 변경 부분 - Google genai, and PGVector
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_postgres.vectorstores import PGVector
# from langchain_community.document_loaders import DirectoryLoader, TextLoader
# import time

import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  
from langchain_openai import ChatOpenAI

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector



In [3]:
#셀프쿼리시 추가 library
from langchain_core.structured_query import Comparator, Operator
from langchain_classic.retrievers.self_query.pgvector import PGVectorTranslator
from langchain_classic.retrievers import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo
from langchain_classic.retrievers import EnsembleRetriever
from kiwipiepy import Kiwi
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain_core.documents import Document

#hybrid 추가 library
from typing import List, Any, Optional
from pydantic import ConfigDict

In [4]:
# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")
    

API 키: sk-proj-
DB connection string : pos


## 임베딩 Ai 연결, DB 연결

In [5]:
# 단계 3: 임베딩(Embedding) 생성
#openai 사용 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

# 임베딩 추가 없이 DB만 호출하고 싶을때는 아래 코드 활성화  
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,  # connection_string이 아니라 connection
)


In [6]:
from sqlalchemy import create_engine, text
#tsvector 추가 전 데이터 확인 


engine = create_engine(CONNECTION_STRING)

with engine.connect() as conn:
    # 1. SI_try collection 확인
    result = conn.execute(text("""
        SELECT uuid 
        FROM langchain_pg_collection 
        WHERE name = 'SI_try'
    """))
    
    collection_row = result.fetchone()
    if collection_row:
        collection_id = collection_row[0]
        
        # 2. SI_try에 저장된 데이터 개수
        result = conn.execute(text("""
            SELECT COUNT(*) 
            FROM langchain_pg_embedding 
            WHERE collection_id = :coll_id
        """), {"coll_id": collection_id})
        
        count = result.fetchone()[0]
        print(f"✅ SI_try에 이미 저장된 데이터: {count:,}개")
        
        # 3. 샘플 데이터 확인
        result = conn.execute(text("""
            SELECT 
                cmetadata->>'no' as 접수번호,
                cmetadata->>'summary' as 요약,
                LEFT(document, 100) as 문서_앞부분
            FROM langchain_pg_embedding 
            WHERE collection_id = :coll_id
            LIMIT 3
        """), {"coll_id": collection_id})
        
        print("\n샘플 데이터:")
        for row in result:
            print(f"  - 접수번호: {row[0]}")
            print(f"    요약: {row[1]}")
            print(f"    문서: {row[2]}...")
            print()
    else:
        print("❌ SI_try collection이 없습니다!")

✅ SI_try에 이미 저장된 데이터: 70,086개

샘플 데이터:
  - 접수번호: 29085
    요약: [식당가6]오더스크린 주문 출력안됨
    문서: [접수번호 29085]
접수일시: 2025-06-10 10:10
가맹점: e 휴게소 고도화 - 죽암(서울)휴게소

문제 요약: [식당가6]오더스크린 주문 출력안됨

상세 내용:
[...

  - 접수번호: 38299
    요약: 라벨 프린터 장애 및 관련 문의
    문서: [접수번호 38299]
접수일시: 2025-07-30 10:32
가맹점: [NEW]나이키 - 신세계여주

문제 요약: 라벨 프린터 장애 및 관련 문의

상세 내용:
010-7777...

  - 접수번호: 41048
    요약: 포스 사용중 멈춤문의
    문서: [접수번호 41048]
접수일시: 2025-08-12 16:56
가맹점: (더샘)한국화장품 - 대전은행지하점(N)

문제 요약: 포스 사용중 멈춤문의

상세 내용:
■ 가맹그룹 :...



In [9]:

engine = create_engine(CONNECTION_STRING)

with engine.connect() as conn:
    # 실제 컬럼 구조 확인
    result = conn.execute(text("""
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'langchain_pg_embedding'
        ORDER BY ordinal_position
    """))
    
    print("=== langchain_pg_embedding 테이블 구조 ===")
    for row in result:
        print(f"  {row[0]}: {row[1]}")

=== langchain_pg_embedding 테이블 구조 ===
  id: character varying
  collection_id: uuid
  embedding: USER-DEFINED
  document: character varying
  cmetadata: jsonb
  search_vector: tsvector


In [10]:

engine = create_engine(CONNECTION_STRING)

def check_current_status():
    """현재 전문검색 설정 상태 확인"""
    
    with engine.connect() as conn:
        # SI_try collection ID
        result = conn.execute(text("""
            SELECT uuid FROM langchain_pg_collection WHERE name = 'SI_try'
        """))
        collection_id = result.fetchone()[0]
        
        # 전체 데이터 수
        result = conn.execute(text("""
            SELECT COUNT(*) 
            FROM langchain_pg_embedding 
            WHERE collection_id = :coll_id
        """), {"coll_id": collection_id})
        total = result.fetchone()[0]
        
        # search_vector가 채워진 데이터 수
        result = conn.execute(text("""
            SELECT COUNT(*) 
            FROM langchain_pg_embedding 
            WHERE collection_id = :coll_id 
              AND search_vector IS NOT NULL
        """), {"coll_id": collection_id})
        filled = result.fetchone()[0]
        
        print("=== 현재 상태 ===")
        print(f"Collection ID: {collection_id}")
        print(f"전체 데이터: {total:,}개")
        print(f"search_vector 채워짐: {filled:,}개")
        print(f"search_vector 비어있음: {total - filled:,}개")
        
        # 인덱스 확인
        result = conn.execute(text("""
            SELECT indexname 
            FROM pg_indexes 
            WHERE tablename = 'langchain_pg_embedding' 
              AND indexname LIKE '%search_vector%'
        """))
        
        indexes = [row[0] for row in result]
        print(f"\n인덱스: {indexes if indexes else '없음'}")
        
        return total, filled, collection_id

# 실행
check_current_status()

=== 현재 상태 ===
Collection ID: ad81cf69-d6ab-431d-9312-7984e3ecdc13
전체 데이터: 70,086개
search_vector 채워짐: 70,086개
search_vector 비어있음: 0개

인덱스: 없음


(70086, 70086, UUID('ad81cf69-d6ab-431d-9312-7984e3ecdc13'))

In [11]:
def fill_search_vector():
    """search_vector 데이터 채우기 (수정 버전)"""
    
    engine = create_engine(CONNECTION_STRING)
    
    with engine.connect() as conn:
        # SI_try collection ID
        result = conn.execute(text("""
            SELECT uuid FROM langchain_pg_collection WHERE name = 'SI_try'
        """))
        collection_id = result.fetchone()[0]
        
        print(f"Collection ID: {collection_id}")
        
        # 비어있는 행 수 확인
        result = conn.execute(text("""
            SELECT COUNT(*) 
            FROM langchain_pg_embedding 
            WHERE collection_id = :coll_id 
              AND search_vector IS NULL
        """), {"coll_id": collection_id})
        
        empty_count = result.fetchone()[0]
        
        if empty_count == 0:
            print("✅ 모든 데이터가 이미 채워져 있습니다!")
            
            # 테스트 검색
            print("\n=== 테스트 검색 ===")
            result = conn.execute(text("""
                SELECT 
                    cmetadata->>'no' as 접수번호,
                    cmetadata->>'summary' as 요약,
                    ts_rank(search_vector, to_tsquery('simple', '배송')) as 점수
                FROM langchain_pg_embedding
                WHERE collection_id = :coll_id
                  AND search_vector @@ to_tsquery('simple', '배송')
                ORDER BY 점수 DESC
                LIMIT 5
            """), {"coll_id": collection_id})
            
            results = result.fetchall()
            print(f"'배송' 검색 결과: {len(results)}개")
            for r in results:
                print(f"  - [{r[0]}] {r[1][:50]}... (점수: {r[2]:.4f})")
            
            return
        
        print(f"\n채워야 할 데이터: {empty_count:,}개")
        print("search_vector 생성 중...")
        
        import time
        start_time = time.time()
        
        # 한 번에 업데이트
        result = conn.execute(text("""
            UPDATE langchain_pg_embedding 
            SET search_vector = to_tsvector('simple', 
                COALESCE(cmetadata->>'summary', '') || ' ' ||
                COALESCE(cmetadata->>'store_name', '') || ' ' ||
                COALESCE(cmetadata->>'main_category', '') || ' ' ||
                COALESCE(cmetadata->>'sub_category', '') || ' ' ||
                COALESCE(document, '')
            )
            WHERE collection_id = :coll_id
              AND search_vector IS NULL
        """), {"coll_id": collection_id})
        
        conn.commit()
        
        elapsed = time.time() - start_time
        print(f"✓ 완료 ({result.rowcount:,}개 행, {elapsed:.1f}초)")
    
    # 인덱스는 별도 연결에서 처리
    print("\n인덱스 확인 중...")
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT COUNT(*) 
            FROM pg_indexes 
            WHERE tablename = 'langchain_pg_embedding' 
              AND indexname LIKE '%search_vector%'
        """))
        
        index_count = result.fetchone()[0]
    
    if index_count == 0:
        print("인덱스 생성 중... (시간이 걸릴 수 있습니다)")
        
        # CONCURRENTLY 사용 시 별도 연결 + AUTOCOMMIT 필요
        with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
            conn.execute(text(f"""
                CREATE INDEX CONCURRENTLY search_vector_si_try_idx 
                ON langchain_pg_embedding 
                USING GIN(search_vector)
                WHERE collection_id = '{collection_id}';
            """))
        
        print("✓ 인덱스 생성 완료")
    else:
        print("✓ 인덱스 이미 존재함")
    
    print("\n✅ 전문 검색 준비 완료!")

# 실행
fill_search_vector()

Collection ID: ad81cf69-d6ab-431d-9312-7984e3ecdc13
✅ 모든 데이터가 이미 채워져 있습니다!

=== 테스트 검색 ===
'배송' 검색 결과: 5개
  - [560] 클레임 배송 문의... (점수: 0.0827)
  - [4122] 감열지 배송 진행상황 문의... (점수: 0.0760)
  - [10101] 직송 배송 알림 번호 변경 문의... (점수: 0.0760)
  - [10182] 배송 요청 실패... (점수: 0.0760)
  - [1895] 감열지 배송 주소 관련 문의... (점수: 0.0760)


# 일반 유사도 검색(기존)

In [12]:
# # 코사인 거리(cosine distance) = 1 - cos(θ)
# # - 범위: 0 ~ 2 (보통 0 ~ 1)
# # - 0에 가까울수록 유사

# #k는 문서 개수, 정의 안하면 기본 4개 
# results = vectorstore.similarity_search_with_score("투썸플레이스 영수증프린터", k=3)
# for i, (doc, score) in enumerate(results, 1):
#     print(f"{i}위 - 유사도: {score:.4f}")
#     print(f"내용: {doc.page_content[:100]}...")
#     print("---")

## Postgre 기존 전문 검색 방식 정의

In [13]:

class PostgreSQLFullTextRetriever(BaseRetriever):
    """PostgreSQL Full Text Search 리트리버"""
    
    # Pydantic V2 방식
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    # 필드 정의
    connection_string: str
    collection_name: str = "SI_try"
    k: int = 5
    collection_id: Optional[Any] = None
    engine: Optional[Any] = None
    
    def __init__(self, connection_string: str, collection_name: str = "SI_try", k: int = 5):
        # Pydantic BaseModel의 __init__ 먼저 호출
        super().__init__(
            connection_string=connection_string,
            collection_name=collection_name,
            k=k
        )
        
        # engine 초기화
        self.engine = create_engine(connection_string)
        
        # collection_id 가져오기
        with self.engine.connect() as conn:
            result = conn.execute(
                text("SELECT uuid FROM langchain_pg_collection WHERE name = :name"),
                {"name": collection_name}
            )
            row = result.fetchone()
            if not row:
                raise ValueError(f"Collection '{collection_name}' not found")
            self.collection_id = row[0]
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        """전문 검색으로 관련 문서 검색"""
        
        search_query = ' & '.join(query.split())
        
        sql = text("""
            SELECT document, cmetadata, 
                   ts_rank(search_vector, to_tsquery('simple', :query)) as rank
            FROM langchain_pg_embedding
            WHERE collection_id = :coll_id
              AND search_vector @@ to_tsquery('simple', :query)
            ORDER BY rank DESC
            LIMIT :k
        """)
        
        with self.engine.connect() as conn:
            results = conn.execute(
                sql, 
                {
                    "query": search_query, 
                    "k": self.k,
                    "coll_id": self.collection_id
                }
            )
            
            documents = []
            for row in results:
                doc = Document(
                    page_content=row[0],
                    metadata=row[1] if row[1] else {}
                )
                documents.append(doc)
            
            return documents
    
    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        return self._get_relevant_documents(query)

print("✅ PostgreSQLFullTextRetriever 정의 완료 (Pydantic V2)")

✅ PostgreSQLFullTextRetriever 정의 완료 (Pydantic V2)


## 하이브리드 리트리버 정의 (벡터+전문)

In [14]:
class HybridRetriever:
    """벡터 검색 + 전문 검색 하이브리드 리트리버"""
    
    def __init__(
        self, 
        vectorstore: PGVector,
        connection_string: str,
        collection_name: str = "SI_try",
        vector_k: int = 5,
        fts_k: int = 5,
        vector_weight: float = 0.5,
        fts_weight: float = 0.5
    ):
        self.vectorstore = vectorstore
        self.connection_string = connection_string
        self.collection_name = collection_name
        self.vector_k = vector_k
        self.fts_k = fts_k
        self.vector_weight = vector_weight
        self.fts_weight = fts_weight
        
        # 벡터 검색 리트리버
        self.vector_retriever = vectorstore.as_retriever(
            search_kwargs={"k": vector_k}
        )
        
        # 전문 검색 리트리버
        self.fts_retriever = PostgreSQLFullTextRetriever(
            connection_string=connection_string,
            collection_name=collection_name,
            k=fts_k
        )
        
        # 앙상블 리트리버 생성
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.vector_retriever, self.fts_retriever],
            weights=[vector_weight, fts_weight]
        )
        
        print(f"✅ Hybrid Retriever 초기화 완료")
        print(f"   - 벡터 검색: top-{vector_k} (가중치: {vector_weight})")
        print(f"   - 전문 검색: top-{fts_k} (가중치: {fts_weight})")
    
    def invoke(self, query: str, **kwargs) -> List[Document]:
        """하이브리드 검색 실행 (invoke 사용)"""
        return self.ensemble_retriever.invoke(query, **kwargs)
    
    def get_relevant_documents(self, query: str, **kwargs) -> List[Document]:
        """하이브리드 검색 실행 (호환성)"""
        return self.invoke(query, **kwargs)

print("✅ HybridRetriever 정의 완료")

✅ HybridRetriever 정의 완료


In [15]:
# ==================== 리트리버 초기화 ====================
# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터스토어 연결
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,
)

# # 하이브리드 리트리버 생성 - 기본 설정
# hybrid_retriever = HybridRetriever(
#     vectorstore=vectorstore,
#     connection_string=CONNECTION_STRING,
#     vector_k=5,          # 벡터 검색 결과 5개
#     fts_k=5,             # 전문 검색 결과 5개
#     vector_weight=0.6,   # 벡터 검색 60%
#     fts_weight=0.4       # 전문 검색 40%
# )

# 하이브리드 리트리버 생성
hybrid_retriever = HybridRetriever(
    vectorstore=vectorstore,
    connection_string=CONNECTION_STRING,
    collection_name="SI_try",
    vector_k=5,
    fts_k=5,
    vector_weight=0.6,
    fts_weight=0.4
)

✅ Hybrid Retriever 초기화 완료
   - 벡터 검색: top-5 (가중치: 0.6)
   - 전문 검색: top-5 (가중치: 0.4)


In [16]:
# # 키워드 검색시 리트리버가 가져오는 문서 확인하는 코드 

# query = "POS 프린터"
# results = hybrid_retriever.invoke(query)

# print(f"\n검색어: '{query}' - 결과 {len(results)}개\n")
# for i, doc in enumerate(results, 1):
#     print(f"{i}. [{doc.metadata.get('no')}] {doc.metadata.get('summary', 'N/A')[:60]}")

In [17]:
# prompt = PromptTemplate.from_template(
#     """You are an assistant for AS call center worker who needs information for error case - answer tasks. 
# Use the following pieces of retrieved context to answer the question. 
# If you don't know the answer, just say that you don't know. 
# Answer in Korean.


# #Context: 
# {context}

# #Question:
# {question}

# #Answer:"""
# )
prompt = PromptTemplate.from_template(
    """You are an assistant for AS call center worker who needs information for error case - answer tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Provide several cases including details and then one summary sentence. 
Answer in Korean.

IMPORTANT: Ignore documents without exact match, Priority is user's query. Strictly follow the query.


#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [43]:
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = (
    {"context": hybrid_retriever.ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# retriever = vectorstore.as_retriever()
# 기본값: search_type="similarity"
# print(hybrid_retriever.search_type)  


In [20]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.

In [44]:
%%time
question = "1월에서 2월 사이 투썸플레이스 키오스크 문의사항을 보여줘."
response = chain.invoke(question)
print(response)

1월과 2월 사이에 투썸플레이스 키오스크 관련 문의 사례는 없습니다.  
현재 제공된 자료에는 1월 31일 부산경성대역점에서 행사 프로모션 설정 관련 문의(접수번호 5394)가 있었으며, 2월에는 해당 기간 내 문의 내용이 기록되어 있지 않습니다.  
요약하자면, 1월과 2월 사이 투썸플레이스 키오스크 관련 문의는 확인되지 않습니다.
CPU times: total: 15.6 ms
Wall time: 1.72 s


In [45]:
%%time
question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어? 상담요약 포함해서 알려줘"
response = chain.invoke(question)
print(response)

투썸 매장에서 POS 프린터 오류 처리 사례는 여러 가지가 있습니다.

1. 강동역점의 경우, 프린터 출력이 흐림, 잘림, 안나옴 현상으로 간헐적 영수증 출력 문제를 접수받았으며, 담당자가 원격으로 확인했으나 정확한 원인 파악이 어려워 매장 직원이 접수 안내를 하도록 조치했습니다.

2. 대림삼거리점에서는 픽업프린터 설정 문제로 출력이 안 되는 상황이었으며, 원격으로 픽업프린터 사용 설정 후 메뉴프린터 연결 방법을 안내했고, 필요 시 다시 연락하도록 조치했습니다.

3. 북대구IC점에서는 프린터가 전원 안 들어오고 출력이 안 되는 문제로 출장 점검이 진행되었으며, 셀프 재설정, 재부팅, 드라이버 재설치 후에도 문제가 해결되지 않아 출장 무상으로 점검을 진행 중입니다.

4. 송산그린시티점에서는 IP 세팅 후 정상화되었으며, 주문서 출력이 안 되는 문제를 원격으로 해결했습니다.

5. 수원나혜석거리점에서는 전원 불이 들어오지 않고 출력이 안 되는 증상으로, 전원 어댑터와 연결 상태를 점검했으나 반응이 없었고, 현재 처리중입니다.

요약하자면, 투썸 매장에서 POS 프린터 오류는 원격 또는 출장 점검을 통해 재설정, 드라이버 재설치, 네트워크 설정 변경 등 다양한 방법으로 해결하려 노력하고 있으며, 일부는 아직 해결 중입니다.
CPU times: total: 31.2 ms
Wall time: 3.6 s


In [23]:
%%time
question = "키오스크 화면이 안 보일 땐 어떻게 해야해?"
response = chain.invoke(question)
print(response)

키오스크 화면이 안 보일 때는 다음과 같은 조치를 취할 수 있습니다.

1. 태블릿 모드 해제: 여러 사례에서 태블릿 모드를 해제하면 화면이 정상으로 복구된 경우가 있습니다. 예를 들어, 북수원정자타운점과 제주오라우정점 사례에서 태블릿 모드 해제 후 문제가 해결되었습니다.

2. 재부팅 또는 재시작: 검은 화면이나 화면이 뜨지 않는 경우, 키오스크를 재부팅하거나 재시작하는 것이 도움이 됩니다. 경산월드컵대로점 사례에서 재부팅 후 정상화된 사례가 있습니다.

3. 디스크 정리 및 임시파일 삭제: 일부 경우에는 임시파일 삭제, 디스크 정리, 디펜더 미사용, 방화벽 해제 등의 원격 조치로 해결 가능합니다. 경산월드컵대로점 사례에서 이러한 방법으로 문제를 해결하였습니다.

4. 화면 닫기 또는 종료: 윈도우 화면이 떠 있는 경우, 화면을 닫거나 종료하는 것도 방법입니다. 제일제당사옥 사례에서 화면을 닫아 해결하였습니다.

요약하자면, 키오스크 화면이 안 보일 때는 태블릿 모드 해제, 재부팅, 임시파일 삭제, 또는 화면 종료 등의 조치를 시도하는 것이 좋습니다.
CPU times: total: 15.6 ms
Wall time: 3.35 s


In [30]:
%%time
question = "iBOX 접속 안될때?"
response = chain.invoke(question)
print(response)

iBOX 접속이 안될 때는 여러 사례가 있습니다.

첫째, 2025년 3월 10일 점촌 나이키 가맹점에서는 iBOX 접속이 안 됐으며, 담당자가 원격으로 전달 후 정상화되어 사용이 가능하게 되었습니다. 상세 내용은 접속 불가 문제였으며, 이후 정상화 처리되었습니다.

둘째, 2025년 3월 3일 G롯데전주 나이키 가맹점에서는 접속 시 모든 아이콘에 에러가 발생했고, 재부팅과 VPN 연결 후 정상 접속이 가능해졌습니다. 이 경우도 원격으로 처리되었습니다.

셋째, 2025년 5월 26일 갤러리아천안 나이키 가맹점에서는 접속이 안 됐으나, 이후 정상 접속되어 사용 안내가 이루어졌습니다.

넷째, 2025년 1월 23일 현대아울렛김포 나이키 가맹점에서는 VPN 연결 후 정상 접속되었으며, 원격으로 해결되었습니다.

다섯째, 2025년 2월 20일 신세계영등포 나이키 가맹점에서는 원격 조회와 네이버 접속이 안 돼 접수된 사례도 있습니다.

요약하자면, iBOX 접속 문제는 네트워크 연결, VPN, 또는 시스템 오류 등 다양한 원인으로 발생하며, 대부분 원격으로 재부팅, VPN 연결, 또는 기타 조치를 통해 해결됩니다.
CPU times: total: 31.2 ms
Wall time: 4.55 s


In [31]:
%%time
question = "스피커 소리 안남?"
response = chain.invoke(question)
print(response)

스피커 소리 안남 문제에 대한 여러 사례를 알려드리겠습니다.

1. CJ 시스템 - 부산 레이카운티점: 포스 오디오로 기본 설정 시 소리 정상이나, 스피커로 기본 설정 후 소리 송출은 되나 소리 안 남. 스피커 확인 안내.
2. CJ 시스템 - 장유신문점: 오디오 재설치 후 동일 증상. 비용 절감을 위해 스피커 연결 추천 후 종료.
3. 투썸플레이스 - 부천중동역점: 소리 설정 변경 후 정상.
4. AtPos 1.0 - 구리점: 스피커 테스트 시 소리 송출 정상이나, 프로그램 내 설정 문제로 인해 스피커 잡히지 않음. 업체 문의 안내.
5. CJ 시스템 - 대방역(2): 오디오 재설치 후 동일 증상, 스피커 확인 안내.
6. KFC 대구문화동: 선재 연결 후 소리 안남, 선 연결 위치 조정 후 정상.
7. e 휴게소 고도화 - LG사이언스파크: 내장 스피커 소리 들리지 않음, 스피커 교체 및 점검 요청 중.
8. 투썸플레이스 - 용인보정점: 스피커 재연결 후 정상.
9. 투썸플레이스 - 안성한경대점: 케이블 재연결 및 포스 재부팅 후 정상.
10. 투썸 국군수도병원점: 스피커 선 재연결 후 정상.

요약하자면, 스피커 소리 안남 문제는 주로 연결 상태 점검, 설정 변경, 또는 스피커 교체를 통해 해결되고 있으며, 일부는 프로그램 내 설정 문제로 인해 발생하는 경우도 있습니다.
CPU times: total: 78.1 ms
Wall time: 4.86 s


In [38]:
%%time
question = "인터넷 접속이 안됨?"
response = chain.invoke(question)
print(response)

인터넷 접속 불량 사례는 여러 가지가 있습니다.

1. 라커룸(AtPos 2.0)에서 인터넷이 안 되어 원격 확인 요청 후 안내받았으며, 원격으로 해결되지 않아 추가 점검이 필요합니다.
2. 영통점(우리생협)에서는 PC 웹 설정 요청 후 설정 완료 및 정상 접속 확인이 이루어졌습니다.
3. 본사(우리생협)에서는 개인 PC 웹 설정 요청 후 설정 완료되어 문제가 해결된 것으로 보입니다.
4. 땡초꼼장어(AtPos 2.0)에서는 포스와 공유기 재부팅 후에도 인터넷이 안 되어 안내를 받았으며, 재부팅 후에도 해결되지 않아 추가 조치가 필요합니다.
5. 대전관평롯데(뚜레쥬르)에서는 정전 이후 포스 결제 불가 및 인터넷 연결 안내가 있었으며, 원격으로 해결 중입니다.
6. G롯데영등포점(KFC)에서는 키오스크 인터넷 연결이 안 되어 재부팅 및 점검 요청 후 처리 중입니다.
7. 신세계광주([NEW]나이키)에서는 인터넷이 안 되어 IBOX 접속이 불가했고, 인터넷 연결 문제로 안내받았습니다.
8. 슈마커이마트천안서북([NEW]나이키)에서는 VPN 소프트웨어 장애로 VPN 접속이 안 되어 재설치 및 점검이 진행 중입니다.

요약하자면, 여러 매장에서 인터넷 접속 문제 또는 관련 장비 장애가 발생했으며, 일부는 원격으로 해결되었거나 추가 점검이 필요한 상태입니다.
CPU times: total: 62.5 ms
Wall time: 3.83 s


In [36]:
%%time
question = "키오스크 화면 안 보임?"
response = chain.invoke(question)
print(response)

키오스크 화면이 보이지 않는 사례는 여러 가지가 있습니다.

1. 안동(춘천)휴게소의 경우, 키오스크 화면이 원격으로는 정상이나 매장에서는 보이지 않는 문제가 발생하였으며, 파워버튼 불이 들어오지 않고 반응이 없었습니다. 이 문제는 방전과 모든 선 재연결 후에도 해결되지 않아 출장 조치가 진행 중입니다.

2. 선산(양평)휴게소의 경우, 키오스크 화면이 검게 보이는 증상이 있었으며, 원격으로는 정상이나 현장 방문 후 보드 교체 등의 조치가 필요하다고 판단되어 처리 중입니다.

3. 투썸플레이스 신도림디큐브시티의 경우, 키오스크 화면이 켜지지 않는 문제가 있었으며, 방전 후 정상으로 복구되었습니다.

이처럼 키오스크 화면이 보이지 않는 문제는 원격 조치로 해결되는 경우도 있지만, 하드웨어 고장이나 연결 문제로 인해 현장 방문이 필요한 경우도 있습니다.
CPU times: total: 15.6 ms
Wall time: 3.39 s


In [37]:
%%time
question = "영수증 출력 안됨?"
response = chain.invoke(question)
print(response)

영수증 출력 문제에 대한 여러 사례를 정리해 드리겠습니다.

1. (주)오뚜기사우회 가맹점에서는 영수증 재발행은 정상이나 결제 영수증 출력만 안 되는 문제가 있었으며, 원격 접속 후 영수증 출력 제어를 설정하여 정상 작동하였습니다.

2. 몽중헌(판교)에서는 영수증 설정 변경 후 출력이 되지 않던 문제를 ENV6 2>8 설정 후 해결하였으며, 원격으로 처리하였습니다.

3. 뚜레_탄현제니스점에서는 버추얼 재설정을 통해 영수증 출력이 정상화되었으며, 두 건 모두 원격으로 해결하였습니다.

4. 일상의틈_동교동홍대입구점에서는 선 재연결 후 정상 출력되었고, 원격으로 처리하였습니다.

5. 통영점에서는 강제 재부팅 후 재접속으로 영수증이 정상 출력되었으며, 원격으로 해결하였습니다.

6. 토니모리(NC백화점송파점)에서는 노트북에서 프린터 USB선 재연결 후 정상 출력되었고, 원격으로 처리하였습니다.

7. [NEW]나이키 롯데월드타워에서는 프린터 재연결 후 설정이 정상임을 확인하였으며, 원격으로 해결하였습니다.

8. KFC홍익대에서는 영수증 출력은 정상화되었으나 카드 결제 시 오류가 있었으며, 리더기 어댑터 재연결 후 정상 작동하였습니다.

요약하자면, 영수증 출력 문제는 프린터 재연결, 설정 변경, 원격 재설정, 또는 장비 재부팅 등 다양한 방법으로 해결되고 있으며, 대부분 원격 처리로 해결 가능합니다.
CPU times: total: 46.9 ms
Wall time: 5.02 s


In [24]:
# #토큰 수 계산 함수 
# import tiktoken
# # 1. 인코더 초기화 (GPT-4o-mini는 cl100k_base 사용)
# encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# # 2. 각 부분의 토큰 수 계산
# def count_tokens(text):
#     return len(encoding.encode(text))

# # 3. 실제 측정
# question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어? 상담요약 포함해서 알려줘."

# # Context 가져오기
# retrieved_docs = hybrid_retriever.ensemble_retriever.invoke(question)
# context = "\n".join([doc.page_content for doc in retrieved_docs])

In [25]:
# # 토큰 수, 가격 계산
# # 모델 마다 토큰 당 가격이 다름 : gpt-4o-mini의 경우 1M token 당 input $0.15	캐시 input $0.075	output$0.60
# # gpt-4o-mini price
# input_price = 0.15/1000000
# output_price = 0.6/1000000

# # 환율
# d2won = 1458

# prompt_tokens = count_tokens(prompt.template)  # 템플릿 고정 부분
# context_tokens = count_tokens(context)
# question_tokens = count_tokens(question)
# output_tokens = count_tokens(response)

# total_input_tokens = prompt_tokens + context_tokens + question_tokens
# total_output_tokens = output_tokens

# total_token_price = total_input_tokens * input_price + output_tokens * output_price
# Kwon_total_token_price = d2won * total_token_price

# print(f"📊 토큰 사용량:")
# print(f"  입력 토큰: {total_input_tokens:,}")
# print(f"    - Prompt: {prompt_tokens}")
# print(f"    - Context: {context_tokens}")
# print(f"    - Question: {question_tokens}")
# print(f"  출력 토큰: {total_output_tokens:,}")
# print(f"  총 토큰: {total_input_tokens + total_output_tokens:,}")
# # 유효숫자 2개
# print(f"총 가격 ($) : {total_token_price:.2g}")
# print(f"총 가격 (원) : {Kwon_total_token_price:.2g}")

In [26]:
# #한달에 드는 총 비용 계산  
# # 주어진 데이터
# monthly_mean = 5840.5  # 한 달 평균 건수
# monthly_std = 651.9    # 표준편차
# questions_per_case = 1  # 1건당 질문 수
# cost_per_question = Kwon_total_token_price  # 1 질문당 비용 (원)

# # 계산
# total_questions_mean = monthly_mean * questions_per_case
# total_questions_std = monthly_std * questions_per_case

# total_cost_mean = total_questions_mean * cost_per_question
# total_cost_std = total_questions_std * cost_per_question

# print("=" * 60)
# print("💰 한 달 LLM 비용 계산")
# print("=" * 60)
# print(f"월 평균 케이스:        {monthly_mean:,.1f}건")
# print(f"1건당 질문 수:         {questions_per_case}번")
# print(f"1질문당 비용:          {cost_per_question}원")
# print()
# print(f"월 총 질문 수:         {total_questions_mean:,.1f}회")
# print(f"                       (±{total_questions_std:,.1f}회)")
# print()
# print(f"💵 월 평균 비용:       {total_cost_mean:,.0f}원")
# print(f"   (±{total_cost_std:,.0f}원)")
# print()
# print(f"예상 범위:")
# print(f"  최소: {total_cost_mean - total_cost_std:,.0f}원")
# print(f"  평균: {total_cost_mean:,.0f}원")
# print(f"  최대: {total_cost_mean + total_cost_std:,.0f}원")
# print()
# print(f"💳 연간 예상 비용:     {total_cost_mean * 12:,.0f}원")
# print(f"                       ({total_cost_mean * 12 / 10000:.1f}만원)")

In [27]:
# # 현재 노트북을 .py로 변환
# !jupyter nbconvert --to python SI_AS_Ensemble.ipynb --output ./python_scripts/SI_AS_Ensemble.py

# 모델별 성능평가

In [50]:
import time
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def get_usd_to_krw():
    """실시간 환율 조회"""
    try:
        response = requests.get("https://api.exchangerate-api.com/v4/latest/USD", timeout=5)
        rate = response.json()['rates']['KRW']
        print(f"💱 현재 환율: 1 USD = {rate:,.0f} KRW")
        return rate
    except:
        rate = 1350
        print(f"💱 환율 조회 실패, 기본값 사용: 1 USD = {rate:,} KRW")
        return rate

def benchmark_chain_models(hybrid_retriever, prompt, question: str, models: list = None):
    
    if models is None:
        models = [
            "gpt-5-mini",
            "gpt-5-nano", 
            "gpt-4o-mini",
            "gpt-3.5-turbo",
            "gpt-4.1-nano",
        ]
    
    # 가격표
    MODEL_PRICING = {
        "gpt-5-mini":                {"input": 0.25, "output": 2},
        "gpt-5-nano":                {"input": 0.05, "output": 0.04},
        "gpt-4o-mini":                {"input": 0.15, "output": 0.6},
        # "gpt-5-codex":           {"input": 1.25, "output": 10},
        "gpt-3.5-turbo":           {"input": 0.5, "output": 1.5},
        "gpt-4.1-nano":           {"input": 0.1, "output": 0.4},
        # "gemini-2.5-flash":             {"input": 0.3, "output": 2.5},
        # "gemini-2.5-flash-lite":           {"input": 0.1, "output": 0.4},
        # "gemini-3-flash-preview":           {"input": 0.5, "output": 3},
    }
    
    # 환율
    usd_to_krw = get_usd_to_krw()
    
    results_summary = []
    
    print(f"\n📝 질문: {question}\n")
    
    for model in models:
        print(f"\n{'='*60}")
        print(f"🤖 모델: {model}")
        print(f"{'='*60}")
        
        try:
            llm = ChatOpenAI(model=model, temperature=0)
            
            # 체인 구성
            chain = (
                {"context": hybrid_retriever.ensemble_retriever, "question": RunnablePassthrough()}
                | prompt
                | llm
                | StrOutputParser()
            )
            
            t_start = time.time()
            response = chain.invoke(question)
            t_elapsed = time.time() - t_start
            
            # 토큰/비용은 콜백으로 따로 측정
            from langchain_community.callbacks import get_openai_callback
            with get_openai_callback() as cb:
                chain_with_cb = (
                    {"context": hybrid_retriever.ensemble_retriever, "question": RunnablePassthrough()}
                    | prompt
                    | llm
                    | StrOutputParser()
                )
                response = chain_with_cb.invoke(question)
            
            input_tokens  = cb.prompt_tokens
            output_tokens = cb.completion_tokens
            total_tokens  = cb.total_tokens
            
            pricing = MODEL_PRICING.get(model, {"input": 0, "output": 0})
            total_cost_usd = (input_tokens / 1000000) * pricing["input"] + \
                             (output_tokens / 1000000) * pricing["output"]
            total_cost_krw = total_cost_usd * usd_to_krw
            
            print(f"  ⏱️  응답시간:  {t_elapsed:.2f}초")
            print(f"  📊 토큰:     입력 {input_tokens:,} / 출력 {output_tokens:,} / 합계 {total_tokens:,}")
            print(f"  💰 비용:     ${total_cost_usd:.6f} / ₩{total_cost_krw:.4f}")
            print(f"\n--- 답변 ---")
            print(response)
            
            results_summary.append({
                "model":         model,
                "time_sec":      round(t_elapsed, 2),
                "input_tokens":  input_tokens,
                "output_tokens": output_tokens,
                "total_tokens":  total_tokens,
                "cost_usd":      round(total_cost_usd, 6),
                "cost_krw":      round(total_cost_krw, 4),
                "response":      response,
            })
            
        except Exception as e:
            print(f"  ❌ 오류: {e}")
            results_summary.append({
                "model":         model,
                "time_sec":      None,
                "input_tokens":  None,
                "output_tokens": None,
                "total_tokens":  None,
                "cost_usd":      None,
                "cost_krw":      None,
                "response":      None,
            })
    
    # 비교 테이블
    print(f"\n\n{'='*75}")
    print(f"📊 모델 비교 요약 (환율 {usd_to_krw:,.0f}원)")
    print(f"{'='*75}")
    print(f"{'모델':<20} {'응답(초)':>8} {'입력토큰':>9} {'출력토큰':>9} {'USD':>10} {'KRW':>10}")
    print(f"{'─'*75}")
    for r in results_summary:
        time_str = f"{r['time_sec']:.2f}"  if r['time_sec']  else "오류"
        in_str   = f"{r['input_tokens']:,}" if r['input_tokens']  else "-"
        out_str  = f"{r['output_tokens']:,}" if r['output_tokens'] else "-"
        usd_str  = f"${r['cost_usd']:.6f}"  if r['cost_usd'] is not None else "-"
        krw_str  = f"₩{r['cost_krw']:.4f}"  if r['cost_krw'] is not None else "-"
        print(f"{r['model']:<20} {time_str:>8} {in_str:>9} {out_str:>9} {usd_str:>10} {krw_str:>10}")
    print(f"{'='*75}")
    
    return results_summary
    


# # 실행
# results = benchmark_chain_models(
#     hybrid_retriever=hybrid_retriever,
#     prompt=prompt,
#     question="영수증이 출력 안될 때 어떻게 조치해야해?"
# )

In [51]:
results = benchmark_chain_models(
    hybrid_retriever=hybrid_retriever,
    prompt=prompt,
    question="영수증이 출력 안될 때 어떻게 조치해야해?"
)

💱 환율 조회 실패, 기본값 사용: 1 USD = 1,350 KRW

📝 질문: 영수증이 출력 안될 때 어떻게 조치해야해?


🤖 모델: gpt-5-mini
  ⏱️  응답시간:  16.19초
  📊 토큰:     입력 1,369 / 출력 1,351 / 합계 2,720
  💰 비용:     $0.003044 / ₩4.1097

--- 답변 ---
사례별 조치 (문서 기준)

1) 영수증이 계속 출력되는 경우  
- 증상: 영수증이 연속 출력됨.  
- 조치: POS 프로그램의 영수증 출력 on/off 설정 방법 안내(설정을 끄거나 켬).  
- 처리 방식: 원격 안내로 해결됨.

2) 결제영수증만 출력되지 않는 경우  
- 증상: 영수증 재발행은 정상이나 결제영수증만 출력 안됨.  
- 조치: 유선 안내로는 해결되지 않아 원격(시트롤) 접속 후 “영수증 출력제어”를 ‘예’로 변경함 → 정상 복구.

3) 영수증 전체가 출력되지 않는 경우(버추얼 관련)  
- 증상: 영수증 전혀 출력 안됨.  
- 조치: 버추얼 프린터 재설정(virtual reset) 수행 → 정상 복구.

4) 설정 변경 후 출력 안되는 경우(환경 변수 관련)  
- 증상: 영수증 설정 변경 후 출력 불가.  
- 조치: ENV6 값을 2 → 8로 변경하여 정상 복구(프로그램 설정 조정).

권장 순서(현장/원격 처리 절차)  
1) POS 프로그램의 영수증 출력 제어(on/off) 확인 및 설정 변경, 2) 문제 지속 시 ENV6 값 확인(필요하면 2→8로 변경), 3) 버추얼 프린터 재설정 수행, 4) 원격 접속하여 ‘영수증 출력제어’ 설정을 ‘예’로 적용 후 재시도, 5) 그래도 안되면 원격지원으로 상세 점검 요청.

요약 한 문장  
영수증 미출력 문제는 우선 POS의 출력제어(on/off) 확인·조정 후(필요시 ENV6 2→8 변경), 버추얼 프린터 재설정이나 원격에서 영수증 출력제어를 ‘예’로 설정하면 대부분 해결됩니다.

🤖 모델: gpt-5-nano
  ⏱️  응답시간:  16